#Function

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Dataset

In [2]:
dataset = pd.read_csv('drive/My Drive/Colab_Test/Dataset/1. Classification/iris.csv')

dataset.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


#Split Data

In [3]:
features = dataset.drop(['Id', 'Species'], axis=1) 
label = dataset['Species'] 

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(features.values)
feature = pd.DataFrame(X, columns=features.columns) #Mengubah settingan feature ke dataframe

In [5]:
from sklearn.model_selection import train_test_split
feature_train, feature_test, label_train, label_test = train_test_split(feature, label, test_size=0.20)

#Machine Learning

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score

In [7]:
logreg = LogisticRegression()
logreg.fit(feature_train, label_train)
prediction = logreg.predict(feature_test)

In [8]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(feature_test, label_test)))
print(f1_score(label_test, prediction, average='macro'))
print(precision_score(label_test, prediction, average='macro'))
print(recall_score(label_test, prediction, average='macro'))

Accuracy of logistic regression classifier on test set: 0.97
0.9620132953466287
0.9761904761904763
0.9523809523809524


#Deploy Model

##Save Deploy Model

In [9]:
import pickle

In [10]:
filename = 'model.pkl'
pickle.dump(logreg, open(filename, 'wb'))

##Load Deploy Model

In [11]:
loaded_model = pickle.load(open(filename, 'rb'))

In [12]:
prediction_loaded = loaded_model.predict(feature_test)

In [13]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(feature_test, label_test)))
print(f1_score(label_test, prediction, average='macro'))
print(precision_score(label_test, prediction, average='macro'))
print(recall_score(label_test, prediction, average='macro'))

Accuracy of logistic regression classifier on test set: 0.97
0.9620132953466287
0.9761904761904763
0.9523809523809524


In [14]:
feature_test

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
131,2.492019,1.726266,1.502013,1.053537
149,0.068662,-0.124958,0.762759,0.790591
50,1.401508,0.337848,0.535296,0.264699
148,0.432165,0.800654,0.933356,1.447956
26,-1.021849,0.800654,-1.227541,-1.050031
12,-1.264185,-0.124958,-1.341272,-1.444450
6,-1.506521,0.800654,-1.341272,-1.181504
35,-1.021849,0.337848,-1.455004,-1.312977
23,-0.900681,0.569251,-1.170675,-0.918558
110,0.795669,0.337848,0.762759,1.053537


In [15]:
test_data = [[-1.506521	, 0.106445	, -1.284407, -1.312977]]

prediction_example = loaded_model.predict(test_data)

prediction_example[0]

'Iris-setosa'

#Web Service

In [16]:
# {
#   "petal_width": -0.173674,
#   "petal_length": -0.587764,
#   "sepal_width": 0.194102,
#   "sepal_length": 0.133226
# }

In [17]:
!pip install flask-ngrok

In [18]:
from flask_ngrok import run_with_ngrok
from flask import Flask

In [19]:
app = Flask(__name__) # Membuat sebuah object Flask

@app.route("/home")
def home():
    return "<h1>Running Flask on Google Colab!</h1>"

In [20]:
from flask import jsonify, request

In [21]:
@app.route('/classify', methods=['POST'])
def classification():
  sepal_width = float(request.json['sepal_width'])
  sepal_length = float(request.json['sepal_length'])
  petal_length = float(request.json['petal_length'])
  petal_width = float(request.json['petal_width'])

  if sepal_width < 0.5 and petal_length > 0.6:
    output_class = "Iris-Versicolor"
  else:
    output_class = "Iris-Virginica"

  return jsonify({
      "output": output_class
  })

In [22]:
run_with_ngrok(app)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d98aec28e3fe.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
